In [1]:
import numpy as np 
import pandas as pd 
# import cudf as pd
import os, gc, re, warnings
warnings.filterwarnings("ignore")

In [ ]:
df0 = pd.read_csv("")

In [2]:
mpn_tr = np.load("/kaggle/input/sentencetf-and-albert-embeddings/train_mpnet_basev2_feats.npy")[:df0.shape[0],:]
alb_tr = np.load("/kaggle/input/sentencetf-and-albert-embeddings/train_albert_basev2_feats.npy")[:df0.shape[0],:]

In [3]:
mpn_te = np.load("/kaggle/input/sentencetf-and-albert-embeddings/test_mpnet_basev2_feats.npy")
alb_te = np.load("/kaggle/input/sentencetf-and-albert-embeddings/test_albert_basev2_feats.npy")

In [4]:
tr_feats = np.concatenate([mpn_tr, alb_tr], axis=1)
del mpn_tr, alb_tr
gc.collect()

150

In [5]:
te_feats = np.concatenate([mpn_te, alb_te], axis=1)
del mpn_te, alb_te
gc.collect()

23

In [6]:
te_feats.shape

(734736, 1536)

In [7]:
tr_feats.shape

(899875, 1536)

In [ ]:
import sys
sys.path.append('../input/iterativestratification')
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
FOLDS = 25
skf = MultilabelStratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=42)
for i,(train_index, val_index) in enumerate(skf.split(dftr,dftr[target_cols])):
    dftr.loc[val_index,'FOLD'] = i
print('Train samples per fold:')
dftr.FOLD.value_counts()

# Train RAPIDS cuML SVR

In [ ]:
from cuml.svm import SVR
import cuml
print('RAPIDS version',cuml.__version__)

In [ ]:
from sklearn.metrics import mean_squared_error

preds = []
scores = []
def comp_score(y_true,y_pred):
    rmse_scores = []
    for i in range(len(target_cols)):
        rmse_scores.append(np.sqrt(mean_squared_error(y_true[:,i],y_pred[:,i])))
    return np.mean(rmse_scores)

#for fold in tqdm(range(FOLDS),total=FOLDS):
for fold in range(FOLDS):
    print('#'*25)
    print('### Fold',fold+1)
    print('#'*25)
    
    dftr_ = dftr[dftr["FOLD"]!=fold]
    dfev_ = dftr[dftr["FOLD"]==fold]
    
    tr_text_feats = all_train_text_feats[list(dftr_.index),:]
    ev_text_feats = all_train_text_feats[list(dfev_.index),:]
    
    ev_preds = np.zeros((len(ev_text_feats),6))
    test_preds = np.zeros((len(te_text_feats),6))
    for i,t in enumerate(target_cols):
        print(t,', ',end='')
        clf = SVR(C=1)
        clf.fit(tr_text_feats, dftr_[t].values)
        ev_preds[:,i] = clf.predict(ev_text_feats)
        test_preds[:,i] = clf.predict(te_text_feats)
    print()
    score = comp_score(dfev_[target_cols].values,ev_preds)
    scores.append(score)
    print("Fold : {} RSME score: {}".format(fold,score))
    preds.append(test_preds)
    
print('#'*25)
print('Overall CV RSME =',np.mean(scores))

In [ ]:
# te_feats = np.concatenate([tfidf_te, xlm_te], axis=1)
# del tfidf_te, xlm_te
# gc.collect()

In [ ]:
# tfidf_tr = tfidf_tr.astype(np.float16)
# tfidf_te = tfidf_te.astype(np.float16)
# gc.collect()

In [ ]:
from sklearn.model_selection import StratifiedKFold
def create_folds(data, num_splits):
    data["kfold"] = -1
    data = data.sample(frac=1).reset_index(drop=True)
    y=data["PRODUCT_TYPE_ID"]
    kf = StratifiedKFold(n_splits=num_splits)
    for f, (t_, v_) in enumerate(kf.split(X=data, y=y)):
        data.loc[v_, 'kfold'] = f
    return data

In [ ]:
dftr = pd.read_csv("/kaggle/input/amazon-ml/dataset/train.csv")
dftr["src"]="train"
dfte = pd.read_csv("/kaggle/input/amazon-ml/dataset/test.csv")
dfte["src"]="test"
print('Train shape:',dftr.shape,'Test shape:',dfte.shape,'Test columns:',dfte.columns)
# df = pd.concat([dftr,dfte],ignore_index=True)

dftr.head()

In [ ]:
print(dftr.shape)
dftr = dftr.dropna(axis=0, subset=['TITLE'])
print(dftr.shape)
dftr.fillna("", inplace=True)
dfte.fillna("", inplace=True)
target_cols = ['PRODUCT_LENGTH']

In [ ]:
train=create_folds(dftr, 5)

In [ ]:
dftr=train.loc[train.kfold.isin([0])]
dftr=dftr.reset_index(drop=True)
display(dftr.head())
dftr.shape

In [ ]:
dftr = dftr[['PRODUCT_ID', 'TITLE', 'PRODUCT_TYPE_ID', 'PRODUCT_LENGTH', 'src', 'kfold']]
dfte = dfte[['PRODUCT_ID', 'TITLE', 'PRODUCT_TYPE_ID', 'src']]

gc.collect()